### 1.  OBJETIVO DO CÓDIGO

Calcular automaticamente quanto você precisa repor de cada produto em cada loja, semana a semana, para nunca faltar estoque.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np


DATA_DIR = Path("data")
SAIDA_DIR = Path("outputs")
SAIDA_DIR.mkdir(exist_ok=True)

ARQUIVO_VENDAS = "forecast.xlsx"
ABA_VENDAS = "BD"
ARQUIVO_ESTOQUE = DATA_DIR / "estoque-total-semana-38-RA.xlsx"
ARQUIVO_CARTEIRA = DATA_DIR / "carteira.xlsx"

SEMANA_INICIO = "W39_25"
SEMANAS_ALVO = 29  # Quantas semanas projetar (janela decrescente)


In [ ]:
def _ordenar_semana(rotulo):
    """Retorna tupla (ano, semana) para permitir ordenação cronológica."""
    semana, ano = rotulo.split("_")
    return int(ano), int(semana[1:])


def carregar_vendas(caminho_arquivo, nome_aba):
    """Carrega o forecast e consolida vendas semanais por FILIAL + SKU."""
    print("Carregando vendas por SKU/PDV...")

    df = pd.read_excel(caminho_arquivo, sheet_name=nome_aba, engine="openpyxl")
    df.columns = df.columns.str.strip()

    df["FILIAL"] = df["FILIAL"].astype(str).str.strip()
    df["FILIAL"] = df["FILIAL"].str.extract(r"(\d+)").fillna("0")
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)

    for coluna in ["PRODUTO", "COR", "TAMANHO", "SKU"]:
        df[coluna] = df[coluna].astype(str).str.upper().str.strip()

    colunas_semana = [col for col in df.columns if col.startswith("202") and "_W" in col]
    rename_map = {}
    for coluna in colunas_semana:
        ano, semana = coluna.split("_W")
        semana = semana.zfill(2)
        rename_map[coluna] = f"W{semana}_{ano[-2:]}"

    df = df.rename(columns=rename_map)
    semanas = sorted(rename_map.values(), key=_ordenar_semana)
    if not semanas:
        raise ValueError("Nenhuma coluna de semana encontrada no arquivo de forecast.")

    colunas_util = ["FILIAL", "SKU", "PRODUTO", "COR", "TAMANHO"] + semanas
    df = df[colunas_util]

    vendas_numericas = df.groupby(["FILIAL", "SKU"], as_index=False)[semanas].sum()
    vendas_texto = df.groupby(["FILIAL", "SKU"], as_index=False)[["PRODUTO", "COR", "TAMANHO"]].first()

    vendas = vendas_texto.merge(vendas_numericas, on=["FILIAL", "SKU"], how="inner")

    print(f"[OK] Vendas: {len(vendas)} SKU/PDV, {len(semanas)} semanas")
    return vendas, semanas


In [ ]:
def _coluna_existente(df, candidatos, descricao):
    for coluna in candidatos:
        if coluna in df.columns:
            return coluna
    raise ValueError(f"Não foi possível localizar a coluna de {descricao}: {df.columns.tolist()}")


def carregar_estoque(caminho_arquivo):
    """Carrega e consolida o estoque atual por FILIAL + SKU."""
    print("Carregando estoque atual por SKU/PDV...")

    df = pd.read_excel(caminho_arquivo, engine="openpyxl")
    df.columns = df.columns.str.strip()

    coluna_filial = _coluna_existente(df, ["FILIAL", "Filial", "Ponto Venda Cód", "Ponto Venda Cod", "PontoVda", "PontoVda Cod"], "filial")
    coluna_estoque = _coluna_existente(df, ["Estoque Total", "Estoque", "Estoque_Total"], "estoque")

    df = df.rename(columns={coluna_filial: "FILIAL", coluna_estoque: "ESTOQUE_ATUAL"})
    if "SKU" not in df.columns:
        raise ValueError("Coluna 'SKU' não encontrada no estoque.")

    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    df["SKU"] = df["SKU"].astype(str).str.upper().str.strip()
    df["ESTOQUE_ATUAL"] = pd.to_numeric(df["ESTOQUE_ATUAL"], errors="coerce").fillna(0)

    df = df.groupby(["FILIAL", "SKU"], as_index=False)["ESTOQUE_ATUAL"].sum()
    print(f"[OK] Estoque consolidado: {len(df)} SKU/PDV")
    return df


def carregar_carteira(caminho_arquivo):
    """Carrega e consolida carteira de pedidos por FILIAL + SKU."""
    print("Carregando carteira por SKU/PDV...")

    df = pd.read_excel(caminho_arquivo, engine="openpyxl")
    df.columns = df.columns.str.strip()

    coluna_filial = _coluna_existente(df, ["FILIAL", "Filial", "PontoVda", "PontoVda Cod", "Ponto Venda Cód", "Ponto Venda Cod"], "filial")
    coluna_pecas = _coluna_existente(df, ["Pecas", "Peças", "Pecas_total", "Pecas Totais"], "pecas")

    df = df.rename(columns={coluna_filial: "FILIAL", coluna_pecas: "CARTEIRA_TOTAL"})
    if "SKU" not in df.columns:
        raise ValueError("Coluna 'SKU' não encontrada na carteira.")

    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int)
    df["SKU"] = df["SKU"].astype(str).str.upper().str.strip()
    df["CARTEIRA_TOTAL"] = pd.to_numeric(df["CARTEIRA_TOTAL"], errors="coerce").fillna(0)

    df = df.groupby(["FILIAL", "SKU"], as_index=False)["CARTEIRA_TOTAL"].sum()
    print(f"[OK] Carteira consolidada: {len(df)} SKU/PDV")
    return df


In [ ]:
def calcular_alvos(df, todas_semanas, semana_inicio, semanas_alvo):
    """Calcula alvos de estoque com janela decrescente."""
    print("Calculando alvos de estoque...")

    if semana_inicio not in todas_semanas:
        raise ValueError(f"Semana inicial {semana_inicio} não encontrada no planejamento de vendas.")

    inicio_idx = todas_semanas.index(semana_inicio)
    fim_idx = inicio_idx + semanas_alvo
    semanas_simulacao = todas_semanas[inicio_idx:fim_idx]
    if not semanas_simulacao:
        raise ValueError("Lista de semanas para simulação está vazia.")

    for posicao, semana in enumerate(semanas_simulacao):
        janela = max(semanas_alvo - posicao, 1)
        idx_semana = todas_semanas.index(semana)
        semanas_para_somar = todas_semanas[idx_semana: idx_semana + janela]
        df[f"ALVO_{semana}"] = df[semanas_para_somar].sum(axis=1)

    return df, semanas_simulacao


In [ ]:
def simular_reposicao(df, semanas_simulacao):
    """Simula reposição semanal garantindo saldo mínimo zero."""
    print("Simulando reposição...")

    reposicoes = {}
    estoques = {}
    estoque_corrente = df["ESTOQUE_INICIAL"].astype(float).to_numpy(copy=True)

    for semana in semanas_simulacao:
        vendas = df[semana].astype(float).to_numpy(copy=True)
        alvo = df[f"ALVO_{semana}"].astype(float).to_numpy(copy=True)

        reposicao = np.maximum(0, alvo - (estoque_corrente - vendas))
        reposicao = np.ceil(reposicao)
        estoque_corrente = np.maximum(0, estoque_corrente - vendas + reposicao)

        reposicoes[f"REPOSICAO_{semana}"] = reposicao.astype(int)
        estoques[f"ESTOQUE_{semana}"] = estoque_corrente.copy()

    for coluna, valores in reposicoes.items():
        df[coluna] = valores
    for coluna, valores in estoques.items():
        df[coluna] = valores

    return df


In [ ]:
def salvar_resultados(df, semanas_simulacao, pasta_saida):
    print("Salvando resultados...")

    dinamicas = {
        "vendas": list(semanas_simulacao),
        "alvos": [f"ALVO_{s}" for s in semanas_simulacao],
        "reposicoes": [f"REPOSICAO_{s}" for s in semanas_simulacao],
        "estoques": [f"ESTOQUE_{s}" for s in semanas_simulacao],
    }

    for coluna in dinamicas["vendas"] + dinamicas["alvos"] + dinamicas["reposicoes"] + dinamicas["estoques"]:
        if coluna not in df.columns:
            df[coluna] = 0

    colunas_exportar = [
        "FILIAL",
        "SKU",
        "PRODUTO",
        "COR",
        "TAMANHO",
        "ESTOQUE_ATUAL",
        "CARTEIRA_TOTAL",
        "ESTOQUE_INICIAL",
    ]
    for grupo in dinamicas.values():
        colunas_exportar.extend(grupo)

    colunas_exportar = [col for col in colunas_exportar if col in df.columns]
    df_final = df[colunas_exportar].copy()

    pasta_saida.mkdir(exist_ok=True)
    destino = pasta_saida / "projecao_sku_pdv.xlsx"
    df_final.to_excel(destino, index=False)

    print(f"[OK] Resultados salvos em: {destino}")
    return df_final


In [ ]:
def main():
    """Executa a simulação completa por SKU/PDV."""
    print("=" * 60)
    print("SIMULAÇÃO DE REPOSIÇÃO POR SKU/PDV")
    print("=" * 60)

    try:
        vendas, todas_semanas = carregar_vendas(ARQUIVO_VENDAS, ABA_VENDAS)
        estoque = carregar_estoque(ARQUIVO_ESTOQUE)
        carteira = carregar_carteira(ARQUIVO_CARTEIRA)

        print()
        print("Combinando dados de vendas, estoque e carteira...")
        dados = pd.merge(vendas, estoque, on=["FILIAL", "SKU"], how="inner")
        dados = pd.merge(dados, carteira, on=["FILIAL", "SKU"], how="left")

        dados["ESTOQUE_ATUAL"] = dados["ESTOQUE_ATUAL"].fillna(0)
        dados["CARTEIRA_TOTAL"] = dados["CARTEIRA_TOTAL"].fillna(0)
        dados["ESTOQUE_INICIAL"] = dados["ESTOQUE_ATUAL"] + dados["CARTEIRA_TOTAL"]

        dados, semanas_simulacao = calcular_alvos(dados, todas_semanas, SEMANA_INICIO, SEMANAS_ALVO)
        dados = simular_reposicao(dados, semanas_simulacao)
        resultado = salvar_resultados(dados, semanas_simulacao, SAIDA_DIR)

        print("=" * 60)
        print("SIMULAÇÃO CONCLUÍDA COM SUCESSO!")
        print(f"Período: {semanas_simulacao[0]} a {semanas_simulacao[-1]}")
        print(f"SKUs analisados: {len(resultado)}")
        print(f"Filiais: {resultado['FILIAL'].nunique()}")
        print("=" * 60)

    except Exception as err:
        print(f"ERRO: {err}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


SIMULAÇÃO DE REPOSIÇÃO POR SKU/PDV
Carregando vendas por SKU/PDV...
